<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
import os

# Specify the path to the downloaded zip file
train_data = r"C:\Users\ARTHUR\Keras Image classification\extracted_data\concrete_data_week4\train"
valid_data = r"C:\Users\ARTHUR\Keras Image classification\extracted_data\concrete_data_week4\valid"
test_data = r"C:\Users\ARTHUR\Keras Image classification\extracted_data\concrete_data_week4\test"

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [2]:
!pip install torchvision

DEPRECATION: Loading egg at c:\users\arthur\appdata\local\programs\python\python311\lib\site-packages\configparser-6.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\arthur\appdata\local\programs\python\python311\lib\site-packages\fxcmpy-1.1.16-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [4]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [7]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [8]:
train_generator = data_generator.flow_from_directory(
    train_data,
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [9]:
validation_generator = data_generator.flow_from_directory(
    valid_data,
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [10]:
model_vgg16 = Sequential()

In [11]:
model_vgg16.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [12]:
model_vgg16.add(Dense(num_classes, activation='softmax'))

In [13]:
model_vgg16.layers

In [14]:
model_vgg16.layers[0].trainable = False

In [15]:
model_vgg16.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [16]:
model_vgg16.layers[0].layers

In [17]:
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)

num_epochs = 2

In [19]:
fit_history = model_vgg16.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2


301/301 [==============================] - 12670s 42s/step - loss: 0.1004 - accuracy: 0.9651 - val_loss: 0.0265 - val_accuracy: 0.9938
Epoch 2/2
301/301 [==============================] - 10187s 34s/step - loss: 0.0191 - accuracy: 0.9954 - val_loss: 0.0158 - val_accuracy: 0.9957


<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [20]:
from keras.models import load_model
resnet_model = load_model(r"C:\Users\ARTHUR\Keras Image classification\classifier_resnet_model.h5")

In [21]:
test_generator = data_generator.flow_from_directory(
    test_data,
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical',
    shuffle=False)

Found 500 images belonging to 2 classes.


In [22]:
# Evaluate the VGG16 model
vgg16_evaluation = model_vgg16.evaluate(test_generator)
print("VGG16 Model Performance:")
print("Test Loss:", vgg16_evaluation[0])
print("Test Accuracy:", vgg16_evaluation[1])

5/5 [==============================] - 120s 24s/step - loss: 0.0116 - accuracy: 0.9980
VGG16 Model Performance:
Test Loss: 0.01161764282733202
Test Accuracy: 0.9980000257492065


In [23]:
test_generator_resnet = data_generator.flow_from_directory(
    test_data,
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical',
    shuffle=False)

Found 500 images belonging to 2 classes.


In [24]:
# Evaluate the ResNet model
resnet_evaluation = resnet_model.evaluate(test_generator_resnet)
print("\nResNet Model Performance:")
print("Test Loss:", resnet_evaluation[0])
print("Test Accuracy:", resnet_evaluation[1])

5/5 [==============================] - 47s 8s/step - loss: 0.3319 - accuracy: 0.8500

ResNet Model Performance:
Test Loss: 0.33190470933914185
Test Accuracy: 0.8500000238418579


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [25]:
# Predict using the VGG16 model
vgg16_predictions = model_vgg16.predict(test_generator)

# Predict using the ResNet model
resnet_predictions = resnet_model.predict(test_generator_resnet)

5/5 [==============================] - 67s 9s/step


In [26]:
# Report class predictions for the first five images
threshold = 0.5
for i in range(5):
    # Determine predicted class based on threshold
    vgg16_prediction = 'Positive' if vgg16_predictions[i][0] > threshold else 'Negative'
    resnet_prediction = 'Positive' if resnet_predictions[i][0] > threshold else 'Negative'
    
    # Print predictions
    print(f"VGG16 Prediction for image {i+1}: {vgg16_prediction}")
    print(f"ResNet Prediction for image {i+1}: {resnet_prediction}")

VGG16 Prediction for image 1: Positive
ResNet Prediction for image 1: Positive
VGG16 Prediction for image 2: Positive
ResNet Prediction for image 2: Positive
VGG16 Prediction for image 3: Positive
ResNet Prediction for image 3: Positive
VGG16 Prediction for image 4: Positive
ResNet Prediction for image 4: Positive
VGG16 Prediction for image 5: Positive
ResNet Prediction for image 5: Positive


In [27]:
import numpy as np

# Initialize counters for positive and negative predictions
positive_count = 0
negative_count = 0

# Iterate through all predictions
for i in range(len(test_generator)):
    # Get batch of test images and true labels
    test_images, true_batch_labels = test_generator[i]
    
    # Predict labels for the batch
    batch_predictions = model_vgg16.predict(test_images)
    
    # Convert predicted probabilities to class labels
    batch_predicted_labels = np.argmax(batch_predictions, axis=1)
    
    # Count positive and negative predictions in the batch
    positive_count += np.sum(batch_predicted_labels == 1)
    negative_count += np.sum(batch_predicted_labels == 0)

# Print the total number of positive and negative predictions
print("Total Positive Predictions:", positive_count)
print("Total Negative Predictions:", negative_count)


4/4 [==============================] - 25s 6s/step
Total Positive Predictions: 249
Total Negative Predictions: 251


In [28]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Assuming y_true and y_pred_labels are obtained from your model
y_pred_labels = (batch_predictions > 0.5).astype(int)  # Thresholding probabilities to get binary labels

# Calculate Metrics
precision = precision_score(true_batch_labels, y_pred_labels, average='micro')
recall = recall_score(true_batch_labels, y_pred_labels, average='micro')
f1 = f1_score(true_batch_labels, y_pred_labels, average='micro')

# Report Results
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Precision: 1.0
Recall: 1.0
F1-score: 1.0


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).